# Generate Schedules

In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
file_path = r"C:\Users\sb013698\Desktop\github\Deep Model Fusion for UBEM\Create Input Data"
file_name = os.path.join(file_path, "best_template.json")

In [3]:
# Read JSON
with open(file_name, "r") as f:
    schedule_raw = json.load(f)

days = {day_schedule["$id"]: day_schedule for day_schedule in schedule_raw["DaySchedules"]}

weeks = {}
for week_schedule in schedule_raw["WeekSchedules"]:
    weeks[week_schedule["$id"]] = week_schedule
    for i, day in enumerate(week_schedule["Days"]):
        weeks[week_schedule["$id"]]["Days"][i] = days[day["$ref"]]

years = {}
for year_schedule in schedule_raw["YearSchedules"]:
    for part in year_schedule["Parts"]:
        part["Schedule"] = weeks[part["Schedule"]["$ref"]]

    years[year_schedule["Name"]] = year_schedule


# Define Dates
# Use a random year
# Use 2018 since 1st of January is on Monday in it
start_year = 2018
end_year = 2018

start_date = f"{start_year}-01-01 00:00:00"
end_date = f"{end_year}-12-31 23:59:59"

date_range = pd.date_range(start=start_date, end=end_date, freq='h')

day_hour_list = date_range.tolist()

df = pd.DataFrame(columns=["Date"] + list(years.keys()), data={"Date": day_hour_list})

df["Year"] = df["Date"].apply(lambda x: x.year)
df["Month"] = df["Date"].apply(lambda x: x.month)
df["Day"] = df["Date"].apply(lambda x: x.day)
df["Hours"] = df["Date"].apply(lambda x: x.hour)
df["WeekDay"] = df["Date"].apply(lambda x: x.weekday())

# Assign Values
for year_name, year_data in years.items():
    df[year_name] = np.nan

    for part in year_data["Parts"]:
        from_month = part["FromMonth"]
        from_day = part["FromDay"]
        to_month = part["ToMonth"]
        to_day = part["ToDay"]
        days = part["Schedule"]["Days"]

        start_date = pd.Timestamp(year=start_year, month=from_month, day=from_day)
        end_date = pd.Timestamp(year=end_year, month=to_month, day=to_day)
        date_range = pd.date_range(start=start_date, end=end_date, freq='h').tolist()

        week_dates = {
            week_day: [date for date in date_range if date.weekday() == week_day]
            for week_day in range(7)
        }

        for week_day, dates in week_dates.items():
            for hour in range(24):
                matching_condition = (df["Date"].isin(dates)) & (df["Hours"] == hour)
                df.loc[matching_condition, year_name] = days[week_day]["Values"][hour]

# Fill NaN values with 0
df = df.fillna(0)

# Save schedules to an Excel file
df.to_excel("schedule.xlsx", index=False, sheet_name="Schedule", engine="openpyxl")

In [4]:
# Read the schedule df and check for missing values
schedule = pd.read_excel("schedule.xlsx")
schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 25 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Date                                          8760 non-null   datetime64[ns]
 1   AlwaysOff_34                                  8760 non-null   int64         
 2   AlwaysOff_34_1                                8760 non-null   int64         
 3   IBB_Year_General_Nat_Vent                     8760 non-null   float64       
 4   IBB_Year_Huzurevi_Otel_DHW                    8760 non-null   float64       
 5   IBB_Year_Huzurevi_Otel_Equipment              8760 non-null   float64       
 6   IBB_Year_Huzurevi_Otel_Heating                8760 non-null   int64         
 7   IBB_Year_Huzurevi_Otel_Lighting               8760 non-null   float64       
 8   IBB_Year_Huzurevi_Otel_Occupancy              8760 non-null   float6

# END